In [1]:
import pandas as pd
from django.db.models import F, Value, CharField, Q, Sum
from django.db.models.functions import Concat
from project.models import Project
from public_data.models import CommunePop, Cerema, CommuneDiff, Scot
from public_data.storages import DataStorage

project = Project.objects.get(id=1486)
project.name

'Diagnostic de CA Grand Auch Coeur de Gascogne'

In [2]:
headers = [
    "Commune",
    "Insee",
    "EPCI",
    "SCoT",
    "Département",
    "Région",
    "Année",
    "Changement",
    "Total",
]
qs = (
    CommunePop.objects.filter(city__in=project.cities.all())
    .annotate(
        Commune=F("city__name"),
        Insee=F("city__insee"),
        EPCI=F("city__epci__name"),
        SCoT=F("city__scot__name"),
        Département=F("city__departement__name"),
        Région=F("city__departement__region__name"),
        Année=F("year"),
        Changement=F("pop_change"),
        Total=F("pop"),
    )
    .values(*headers)
)
qs[:10]

<QuerySet [{'Commune': 'Leboulin', 'Insee': '32207', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2019, 'Changement': 1, 'Total': 342}, {'Commune': 'Leboulin', 'Insee': '32207', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2018, 'Changement': -5, 'Total': 341}, {'Commune': 'Leboulin', 'Insee': '32207', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2017, 'Changement': -3, 'Total': 346}, {'Commune': 'Leboulin', 'Insee': '32207', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2016, 'Changement': -3, 'Total': 349}, {'Commune': 'Leboulin', 'Insee': '32207', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Année': 2015, 'Changement':

In [3]:
df = (
    pd.DataFrame(qs, columns=headers)
    .fillna(0.0)
    .pivot(columns=["Année"], index=headers[:6], values=["Changement", "Total"])
)
df

Changement  \
Année                                                                                              2006   
Commune              Insee EPCI                            SCoT        Département Région                 
Antras               32003 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Auch                 32013 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Augnax               32014 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Auterive             32019 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Ayguetinte           32024 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Biran                32054 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Bonas                32059 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castelnau-Barbarens  32076 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castillon-Massas     32089 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castin               32091 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castéra-Verduzan     32083 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Crastes              32112 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Duran                32117 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Jegun                32162 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Lahitte              32183 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Lavardens            32204 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Leboulin             32207 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Mirepoix             32258 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Montaut-les-Créneaux 32279 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Montégut             32282 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Mérens               32251 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Nougaroulet          32298 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Ordan-Larroque       32301 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Pavie                32307 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Pessan               32312 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Peyrusse-Massas      32316 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Preignan             32331 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Puycasquier          32335 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Roquefort            32347 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Roquelaure           32348 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Saint-Jean-Poutge    32382 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Saint-Lary           32384 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Sainte-Christie      32368 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Tourrenquets         32453 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   

                                                                                                     \
Année                         

In [4]:
headers = [
    "Commune",
    "Insee",
    "EPCI",
    "SCoT",
    "Département",
    "Région",
    "Année",
    "Changement",
    "Total",
]
qs = (
    CommunePop.objects.filter(city__in=project.cities.all())
    .annotate(
        Commune=F("city__name"),
        Insee=F("city__insee"),
        EPCI=F("city__epci__name"),
        SCoT=F("city__scot__name"),
        Département=F("city__departement__name"),
        Région=F("city__departement__region__name"),
        Année=F("year"),
        Changement=F("household_change"),
        Total=F("household"),
    )
    .values(*headers)
)
df = (
    pd.DataFrame(qs, columns=headers)
    .fillna(0.0)
    .pivot(columns=["Année"], index=headers[:6], values=["Changement", "Total"])
)
df

Changement  \
Année                                                                                              2006   
Commune              Insee EPCI                            SCoT        Département Région                 
Antras               32003 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Auch                 32013 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Augnax               32014 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Auterive             32019 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Ayguetinte           32024 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Biran                32054 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Bonas                32059 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castelnau-Barbarens  32076 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castillon-Massas     32089 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castin               32091 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Castéra-Verduzan     32083 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Crastes              32112 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Duran                32117 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Jegun                32162 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Lahitte              32183 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Lavardens            32204 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Leboulin             32207 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Mirepoix             32258 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Montaut-les-Créneaux 32279 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Montégut             32282 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Mérens               32251 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Nougaroulet          32298 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Ordan-Larroque       32301 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Pavie                32307 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Pessan               32312 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Peyrusse-Massas      32316 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Preignan             32331 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Puycasquier          32335 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Roquefort            32347 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Roquelaure           32348 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Saint-Jean-Poutge    32382 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Saint-Lary           32384 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Sainte-Christie      32368 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   
Tourrenquets         32453 CA Grand Auch Coeur de Gascogne De gascogne Gers        Occitanie        0.0   

                                                                                                   \
Année                           

In [5]:
config = [
    {"name": "Insee", "db": F("city__insee"), "type": "index"},
    {"name": "Commune", "db": F("city__name"), "type": "index"},
    {"name": "EPCI", "db": F("city__epci__name"), "type": "index"},
    {"name": "SCoT", "db": F("city__scot__name"), "type": "index"},
    {
        "name": "Département",
        "db": F("city__departement__name"),
        "type": "index",
    },
    {
        "name": "Région",
        "db": F("city__departement__region__name"),
        "type": "index",
    },
    {
        "name": "Plus_ancien_millésime",
        "db": Value(project.first_year_ocsge),
        "type": "index",
    },
    {
        "name": "Plus_récent_millésime",
        "db": Value(project.last_year_ocsge),
        "type": "index",
    },
    {
        "name": "Différentiel",
        "db": Value(project.last_year_ocsge),
        "type": "columns",
    },
]
qs = (
    CommuneSol.objects.filter(
        city__in=project.cities.all(), matrix__is_artificial=True, year=project.last_year_ocsge
    )
    .annotate(**{_["name"]: _["db"] for _ in config})
    .values(*[_["name"] for _ in config])
    .annotate(Surface_artificielle_dernier_millésime=Sum("surface"))
    .order_by("Insee")
)
df = (
    pd.DataFrame(
        qs,
        columns=[
            *[_["name"] for _ in config],
            "Surface_artificielle_dernier_millésime",
        ],
    )
    .fillna(0.0)
    .pivot(
        columns=[_["name"] for _ in config if _["type"] == "columns"],
        index=[_["name"] for _ in config if _["type"] == "index"],
        values=["Surface_artificielle_dernier_millésime"],
    )
)
df

,,,,,,,,Surface_artificielle_dernier_millésime
,,,,,,,Différentiel,2019
Insee,Commune,EPCI,SCoT,Département,Région,Plus_ancien_millésime,Plus_récent_millésime,
32003,Antras,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,17.3000
32013,Auch,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1702.3100
32014,Augnax,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,24.2800
32019,Auterive,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,93.0400
32024,Ayguetinte,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,39.5100
32054,Biran,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,127.8400
32059,Bonas,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,67.4800
32076,Castelnau-Barbarens,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,157.2300


In [7]:
total_artificielle = sum(_["Surface_artificielle_dernier_millésime"] for _ in qs)
total_artificielle

Decimal('5000.6700')

In [8]:
config[8]["db"] = Concat(
    "year_old", Value("-"), "year_new", output_field=CharField()
)
qs2 = (
    CommuneDiff.objects.filter(
        city__in=project.cities.all(),
        year_old__gte=project.analyse_start_date,
        year_old__lte=project.analyse_end_date,
    )
    .annotate(
        **{_["name"]: _["db"] for _ in config},
        Artificialisation=F("new_artif"),
        Renaturation=F("new_natural"),
        Artificialisation_nette=F("net_artif"),
    )
    .values(
        *[_["name"] for _ in config],
        "Artificialisation",
        "Renaturation",
        "Artificialisation_nette",
    )
)
qs2[:3]

<QuerySet [{'Insee': '32207', 'Commune': 'Leboulin', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Plus_ancien_millésime': 2016, 'Plus_récent_millésime': 2019, 'Différentiel': '2016-2019', 'Artificialisation': Decimal('0.3900'), 'Renaturation': Decimal('0.0200'), 'Artificialisation_nette': Decimal('0.3700')}, {'Insee': '32014', 'Commune': 'Augnax', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Plus_ancien_millésime': 2016, 'Plus_récent_millésime': 2019, 'Différentiel': '2016-2019', 'Artificialisation': Decimal('1.2700'), 'Renaturation': Decimal('0.0600'), 'Artificialisation_nette': Decimal('1.2100')}, {'Insee': '32089', 'Commune': 'Castillon-Massas', 'EPCI': 'CA Grand Auch Coeur de Gascogne', 'SCoT': 'De gascogne', 'Département': 'Gers', 'Région': 'Occitanie', 'Plus_ancien_millésime': 2016, 'Plus_récent_millésime': 2019, 'Différentiel': '2016-2019', 'Art

In [9]:
df2 = (
    pd.DataFrame(
        qs2,
        columns=[
            *[_["name"] for _ in config],
            "Artificialisation",
            "Renaturation",
            "Artificialisation_nette",
        ],
    )
    .fillna(0.0)
    .pivot(
        columns=[_["name"] for _ in config if _["type"] == "columns"],
        index=[_["name"] for _ in config if _["type"] == "index"],
        values=[
            "Artificialisation",
            "Renaturation",
            "Artificialisation_nette",
        ],
    )
)
df2

,,,,,,,,Artificialisation,Renaturation,Artificialisation_nette
,,,,,,,Différentiel,2016-2019,2016-2019,2016-2019
Insee,Commune,EPCI,SCoT,Département,Région,Plus_ancien_millésime,Plus_récent_millésime,,,
32003,Antras,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,0.0000,0.0100,-0.0100
32013,Auch,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,25.7900,2.1500,23.6400
32014,Augnax,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1.2700,0.0600,1.2100
32019,Auterive,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1.2500,0.0000,1.2500
32024,Ayguetinte,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,0.0900,0.0000,0.0900
32054,Biran,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1.2300,0.4100,0.8200
32059,Bonas,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,0.0300,0.0000,0.0300
32076,Castelnau-Barbarens,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1.1500,0.0200,1.1300


In [10]:
df4 = df.merge(df2, left_index=True, right_index=True)
df4

,,,,,,,,Surface_artificielle_dernier_millésime,Artificialisation,Renaturation,Artificialisation_nette
,,,,,,,Différentiel,2019,2016-2019,2016-2019,2016-2019
Insee,Commune,EPCI,SCoT,Département,Région,Plus_ancien_millésime,Plus_récent_millésime,,,,
32003,Antras,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,17.3000,0.0000,0.0100,-0.0100
32013,Auch,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,1702.3100,25.7900,2.1500,23.6400
32014,Augnax,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,24.2800,1.2700,0.0600,1.2100
32019,Auterive,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,93.0400,1.2500,0.0000,1.2500
32024,Ayguetinte,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,39.5100,0.0900,0.0000,0.0900
32054,Biran,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,127.8400,1.2300,0.4100,0.8200
32059,Bonas,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,67.4800,0.0300,0.0000,0.0300
32076,Castelnau-Barbarens,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,157.2300,1.1500,0.0200,1.1300


In [ ]:
df3 = df.copy()
df3 = df3.astype(float)
df3

In [ ]:
config = [
    {"name": "Commune", "db": "city__name", "type": "index"},
    {"name": "Insee", "db": "city__insee", "type": "index"},
    {"name": "EPCI", "db": "city__epci__name", "type": "index"},
    {"name": "SCoT", "db": "city__scot__name", "type": "index"},
    {"name": "Département", "db": "city__departement__name", "type": "index"},
    {
        "name": "Région",
        "db": "city__departement__region__name",
        "type": "index",
    },
    {"name": "Année", "db": "year", "type": "columns"},
    {
        "name": "Code",
        "db": "matrix__couverture__code_prefix",
        "type": "columns",
    },
    {
        "name": "Libellé",
        "db": "matrix__couverture__label_short",
        "type": "columns",
    },
]
qs1 = (
    CommuneSol.objects.filter(
        city__in=project.cities.all(),
        year__gte=project.analyse_start_date,
        year__lte=project.analyse_end_date
    )
    .annotate(**{_["name"]: F(_["db"]) for _ in config})
    .values(*[_["name"] for _ in config])
    .annotate(surface=Sum("surface"))
    .order_by(*[_["name"] for _ in config])
)
config[7] = {
    "name": "Code",
    "db": "matrix__usage__code_prefix",
    "type": "columns",
}
config[8] = {
    "name": "Libellé",
    "db": "matrix__usage__label_short",
    "type": "columns",
}
qs2 = (
    CommuneSol.objects.filter(
        city__in=project.cities.all(),
        year__gte=project.analyse_start_date,
        year__lte=project.analyse_end_date,
    )
    .annotate(**{_["name"]: F(_["db"]) for _ in config})
    .values(*[_["name"] for _ in config])
    .annotate(surface=Sum("surface"))
    .order_by(*[_["name"] for _ in config])
)
qs = qs1.union(qs2)
qs.count()

In [ ]:
df = (
    pd.DataFrame(qs, columns=["surface"] + [_["name"] for _ in config])
    .pivot(
        columns=[_["name"] for _ in config if _["type"] == "columns"],
        index=[_["name"] for _ in config if _["type"] == "index"],
        values=["surface"],
    )
    .fillna(0.0)
)
cols = sorted(list(df.columns.values), key=lambda x: f"{x[1]}{x[2]}")
df = df[cols]
df

In [ ]:
list(df.columns.values)